
<center>
    <h1> INF285 - Computación Científica </h1>
    <h1> Tarea N°1, V0.3 </h1>
    
</center>

<p>

</p>

## Instrucciones

* La tarea es individual, sin embargo se invita a todos l@s estudiantes a que discutan entre ustedes las preguntas pero luego implementen de forma individual su tarea.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
* **No modifique la firma de las funciones** (a menos que se le diga lo contrario) y respete el output que se le exije. **En caso de no respetar esta regla la función se considerará errónea.**
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el jueves 6 de mayo a las **18:00 hrs**.  
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).
* Puede agregar funciones extras siempre y cuando **no interfieran en las firmas** de las funciones principales establecidas.

## Introducción 

En esta primera tarea de INF-285, versión 2021-1, estudiaremos la importancia de los primeros temas estudiados en el curso, los cuales son: Representación de Punto Flotante, Pérdida de Importancia, Errores de Cancelación y Búsqueda de Ceros. 
El desarrollo de cada uno de esos temas se presenta en una serie de preguntas donde deberá ir decidiendo, pregunta a pregunta, cómo cada uno de los temas se aplica.
En general, los temas no se analizan por separado, sino de manera acoplada. 
Es muy importante que cada uno de los problemas sea analizado teóricamente primero, para luego poner su conocimiento en acción. 
Cada problema puede ser desarrollado de diversas formas, sin embargo, es muy importante determinar al final si el camino elegido resuelve la pregunta presentada.
Es decir, debe aplicar la metodología IDEA!

## Problemas

### 1. Simulador (60 ptos) 

Dada la familia de polinomios de grado 3 con la forma:


\begin{equation}
f(x)=-1 + x + 2 A^2 x - 3 A^2 x^2 + A^2 x^3
\end{equation}

Se pide implementar un algoritmo que reciba como parámetros el valor de $A$ (con $|A|>>1$), 'bits_mant' que corresponde a la cantidad de bits que tiene la mantisa y 'bits_exp' que corresponde a la cantidad de bits que tiene el exponente. 
El algoritmo se ejecutará en un computador ficticio que representa los números con esa cantidad de bits para el estándar de punto flotante IEEE. 
Debe controlar los casos especiales correctamente.
Esta implementación debe calcular las raíces de $f$ sin pérdida de **importancia** con la cantidad de bits disponibles para la mantisa y la cantidad de bits para el exponente.
Para obtener las raíces de $f$ usted debe encontrar de forma algebraica sus raíces y luego proponer un algoritmo basado en las _fórmulas_ obtenidas.

El computador ficticio cuenta con las operaciones matemáticas necesarias para obtener las raíces. 
Considere como límites de bits para la mantisa en el intervalo $[2,52]$ y en el intervalo $[2,11]$ para los bits del exponente.

Ejemplo:

```python
# Algoritmo de ejemplo
a = 9819824.624837
b = 148736.523476
c = a+b

# Implementación de ejemplo con Representación de Punto Flotante de 'bits_mant' bits en la mantisa y 'bits_exp' bits en el exponente.
am = f_new_rep(9819824.624837,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
bm = f_new_rep(148736.523476,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
cm = f_suma(am,bm,bits_mant,bits_exp) # Aproximar el output de la suma y cada operación en la nueva representación.
```


In [38]:
import numpy as np
import bitstring as bs

In [39]:
def to_binary(f):
    #Funcion de los archivos del curso para convertir un numero flotante en IEEE 754
    b = bs.pack('>d', f)
    b = b.bin
    return [b[0], b[1:12] ,b[12:]]

def approx(mant, other_bits):
    #Función para aproximar la mantisa
    if other_bits[0] == '0':
        #print("caso 1")
        return mant
    
    if other_bits[0] == '1':
        if '1' in other_bits[1:]:
            #print("caso 2")
            return bin(int(mant,2) + 1)[2:]
        else:
            if mant[(len(mant) - 1)] == '1':
                #print("caso 3")
                mant = bin(int(mant,2) + 1)[2:]
                return mant
            else:
                #print("caso 4")
                return mant
            
def get_new_mant(x, bits_mant):
    #Funión que retorna la mantisa ya aproximada con una cantidad de bits especifica (Retorno en binario)
    original = to_binary(x)
    o_mant = original[2]
    new_mant = ''
    other_bits = ''
    
    for b in range(52):
        if (b < bits_mant):
            new_mant = new_mant + o_mant[b]
        else:
            other_bits = other_bits + o_mant[b]
    
    mant_approx = new_mant
    
    if (len(other_bits) != 0):
        mant_approx = approx(new_mant, other_bits)
        
    return mant_approx

def approx_exp(int_exp, new_exp, bits_exp):
    #Función para aproximar el exponente
    if len(new_exp[2:]) <= bits_exp:
        return int_exp
    else:
        return float('inf')

def get_new_exp(x, bits_exp):
    #Función que retorna el exponente aproximado con una cantidad de bits especifica (Retorno en float)
    original = to_binary(x)
    o_exp = int(original[1],2) - 1023
    bias = 2**(bits_exp - 1) - 1
    
    new_exp = o_exp + bias
    new_exp_approx = approx_exp(new_exp, bin(new_exp), bits_exp)
    
    return new_exp_approx

def get_new_num(new_mant, new_exp, bits_exp):
    #Función que retorna un número en float dado un numero en IEEE 754 
    
    one_point = '1' + new_mant
    
    float_part = 0.0
    
    bias = 2**(bits_exp - 1) - 1
    exp = new_exp - bias
    
    for b in range(len(one_point)):
        if one_point[b] == '1':
            float_part = float_part + 2**(b*(-1))
            
    new_num = (float_part)*(2**exp)
    return new_num

### Desarrollo de las formulas

La función original es:

\begin{equation}
f(x)=-1 + x + 2 A^2 x - 3 A^2 x^2 + A^2 x^3
\end{equation}

La cual tiene una raiz independiente de A, que es 1. Al factorizar por (x - 1), se tiene que:

\begin{equation}
f(x)= (x-1) (A^2 x^2 - 2 A^2 x + 1)
\end{equation}

De esta forma, podemos centrarnos en la segunda ecuación, que es cuadrática. Al utilizar la fórmula para calcular sus raices queda que:

\begin{equation}
X_1 =  \frac{A^2 -\sqrt{(A^4 - A^2)}}{A^2}
\end{equation}

\begin{equation}
X_2 =  \frac{A^2 +\sqrt{(A^4 - A^2)}}{A^2}
\end{equation}

Por otra parte, consideremos el siguiente "uno conveniente": 

\begin{equation}
\frac{A^2 + \sqrt{(A^4 - A^2)}} {A^2 +\sqrt{(A^4 - A^2}}
\end{equation}

Al multiplicar lo anterior por $X_1$ se obtiene lo siguiente:

\begin{equation}
X_1 =  \frac{A^2 -\sqrt{(A^4 - A^2)}}{A^2} \frac{A^2 + \sqrt{(A^4 - A^2)}} {A^2 +\sqrt{(A^4 - A^2}}
\end{equation}

\begin{equation}
X_1 =  \frac{A^4 - (A^4 - A^2)}{A^4 + A^2 \sqrt{(A^4 - A^2)}}
\end{equation}

\begin{equation}
X_1 =  \frac{A^2}{A^4 + A^3 \sqrt{(A^2 - 1)}}
\end{equation}

Por otra parte, dado que $|A|>>1$, podemos decir que $\sqrt{(A^2 - 1)} \approx A$ , por lo tanto:

\begin{equation}
X_1 =  \frac{1}{2 A^2}
\end{equation}

Además, al sumar las raices de las ecuaciones cuadráticas, se tiene la siguiente identidad:

\begin{equation}
X_1 + X_2 =  \frac{-b}{a}
\end{equation}

Es decir:

\begin{equation}
\frac{1}{2 A^2} + X_2 =  2
\end{equation}

\begin{equation}
X_2 =  2 - \frac{1}{2 A^2}
\end{equation}

Por lo tanto, el valor de nuestras 3 raices está dado por:

\begin{equation}
X_1 =  1
\end{equation}

\begin{equation}
X_2 =  \frac{1}{2 A^2}
\end{equation}

\begin{equation}
X_3 =  2 - \frac{1}{2 A^2}
\end{equation}



In [49]:
"""
input
x         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_new_rep(x,bits_mant,bits_exp):
    # Algoritmo de representación de punto flotante modificada.
    sign = 1
    if (x < 0):
        sign = -1
    
    new_exp = get_new_exp(x, bits_exp)
    new_mant = get_new_mant(x, bits_mant)
    
    result = get_new_num(new_mant, new_exp, bits_exp)
    
    print(new_mant)
    print(new_exp)
    
    if new_exp == float('inf'):
        if '1' in new_mant:
            return np.nan
        else:
            return float('inf')*sign
        
    else:
        return  result*sign

"""
input
n1        : (double) valor a sumar
n2        : (double) valor a sumar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_suma(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la suma de dos numeros manteniendo el estandar asignado
    n_sumado = n1 + n2
    result = f_new_rep(n_sumado, bits_mant, bits_exp)
    
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_multiplicacion(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la multiplicación de dos numeros manteniendo el estandar asignado
    n_multiplicado = n1*n2
    result = f_new_rep(n_multiplicado, bits_mant, bits_exp)
    
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_division(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la división de dos numeros manteniendo el estandar asignado
    n_dividido = n1/n2
    result = f_new_rep(n_dividido, bits_mant, bits_exp)
    
    return result

"""
input
A         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
x_roots      : (array) raices del polinomio ordenadas de menor a mayor
"""
def f_find_roots(A,bits_mant,bits_exp):
    # Algoritmo para encontrar raíces de f(x).
    # f = lambda x: (x - 1)*((A**2)*(x - 2)*x + 1)
    
    A_new_rep = f_new_rep(A, bits_mant, bits_exp)
    u_aux =  f_multiplicacion(2.0, A_new_rep ,bits_mant, bits_exp)
    u = f_multiplicacion(u_aux, A_new_rep ,bits_mant, bits_exp)
    
    r1 = 1.0
    r2 = f_division(1.0, u, bits_mant,bits_exp)
    
    arg1 = f_division(1.0, A_new_rep, bits_mant, bits_exp)
    arg2 = f_multiplicacion(2.0, A_new_rep ,bits_mant, bits_exp)
    arg3 = f_division(1.0, arg2, bits_mant, bits_exp)
    arg4 = f_multiplicacion(-1.0, arg3 ,bits_mant, bits_exp)
    
    p1 = f_suma(2.0, arg1, bits_mant, bits_exp)
    p2 = f_suma(1.0, arg4, bits_mant, bits_exp)
    
    r3 = f_multiplicacion(p1, p2 ,bits_mant, bits_exp)
    
    x_roots = [r1, r2, r3]
    
    x_roots.sort()
    return x_roots

### Sinusoidal de Oreman (40 puntos)
Considere la siguiente función en 3 variables:
\begin{equation}
T(t,\alpha,\beta) = \alpha\cdot \cos(t)+\beta\cdot \sin(\log(t)).
\end{equation}

La cual permite calcular la temperatura de una cámara de ambiente dado los parámetros $\alpha$ y $\beta$ los cuales corresponden a las condiciones iniciales del experimento. 
Por otro lado $t$ representa el tiempo en días transcurridos desde que se inició.

A usted le permiten utilizar la cámara por un tiempo limitado, para esto planea terminar a los 25 días pero debe asegurarse que las sustancias sean extraídas a temperatura 0.

Cree un programa que reciba los parametros $\alpha$ y $\beta$ y que retorne el día en el cual deba ser extraído el material cumpliendo la condición de ser el día más cercano a 25 (ya sea antes o después).


In [51]:
def newton_method(f, fp, x0, rel_error=1e-8, m=1, maxNumberIterations=100):
    #Initialization of hybrid error and absolute
    hybrid_error = 100
    error_i = np.inf
    
    #print('i |     x_i      |     x_{i+1}  |     |x_{i+1}-x_i|      |  e_{i+1}/e_i | e_{i+1}/e_i^2')
    #print('----------------------------------------------------------------------------------------')
    
    #Iteration counter
    i = 1
    while (hybrid_error > rel_error and hybrid_error < 1e12 and i<=maxNumberIterations):
        #Newton's iteration
        x1 = x0-m*f(x0)/fp(x0)
        
        #Checking if root was found
        if f(x1) == 0.0:
            hybrid_error = 0.0
            break
        
        #Computation of hybrid error
        hybrid_error = abs(x1-x0)/np.max([abs(x1),1e-12])
        
        #Computation of absolute error
        error_iminus1 = error_i
        error_i = abs(x1-x0)
        
        #Increasing counter
        i += 1
        
        #Showing some info
        #print("%d | %.10f | %.10f | %.20f | %.10f | %.10f" % 
              #(i, x0, x1, error_i, error_i/error_iminus1, error_i/(error_iminus1**2)))
        #Updating solution
        x0 = x1
    
    #Checking if solution was obtained
    if hybrid_error < rel_error: 
        return x1
    elif i>=maxNumberIterations:
        #print('Newton''s Method did not converged. Too many iterations!!')
        return None
    else:
        #print('Newton''s Method did not converged!')
        return None

"""
input
alpha: (float) parametro alpha
beta: (float) parametro beta
tol: (float) tolerancia
output
root: (double) raiz obtenida
"""

def raiz_de_oreman(alpha, beta, tol = 1e-10):
    # Su implementación.
    
    if (alpha == 0 and beta == 0):
        return ("Existen infinitas soluciones.")

    #T(t, alpha, beta)
    T = lambda x: alpha*(np.cos(x)) + beta*(np.sin(np.log(x)))  
    #Derivada de T
    Tp = lambda x: beta*(np.cos(np.log(x))/x) - alpha*(np.sin(x))
    
    root1 = newton_method(T, Tp, 24, tol)
    root2 = newton_method(T, Tp, 26, tol)
    
    if root1 is None and root2 is None:
        root = "No hay raices"
    
    if root2 is None:
        root = root1
        
    elif root1 is None:
        root = root2
    
    else:
        if ((25 - root1) < (root2 - 25)):
            root = root1
        else:
            root = root2
    
    return root

alpha = float(input("Enter alpha: \n"))
beta = float(input("Enter beta: \n"))
print("\n")
raiz_de_oreman(alpha, beta)

Enter alpha: 
1
Enter beta: 
6




25.94817644647704

# Referencias

to_binary: Función obtenida de los archivos del curso. Utilizada en la pregunta 1.

newton_method: Función obtenida de los archivos del curso. Utilizada en la pregunta 2. 